In [9]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  !mkdir src
  !cp -r drive/MyDrive/IFT780/HappyWhale/src/* src/
  !mkdir data
  !cp -r drive/MyDrive/IFT780/HappyWhale/data/* data/
  IN_COLAB = True
except:
  IN_COLAB = False
  #%pip install -r requirements.txt

# Pour automatiquement recharger les modules externes
# voir http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of src.DataManager failed: Traceback (most recent call last):
  File "C:\Users\gaeta\anaconda3\envs\forage\lib\site-packages\IPython\extensions\autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\gaeta\anaconda3\envs\forage\lib\site-packages\IPython\extensions\autoreload.py", line 455, in superreload
    module = reload(module)
  File "C:\Users\gaeta\anaconda3\envs\forage\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "c:\Users\gaeta\OneDrive\Documents\Developpement\Python\Forage\Projet\HappyWhale\src\DataManager.py", line 8, in <module>
    from src.transformation import Transformation
  File "c:\Users\gaeta\OneDrive\Documents\Developpement\Python\Forage\Projet\HappyWhale\src\tr

# HappyWhale Challenge - Team WhalePlayed

This notebook presents the work done by Gaétan Rey, Julien Levarlet and Timothée Wright, as part of the challenge https://www.kaggle.com/competitions/happy-whale-and-dolphin/overview .

Imports :

In [10]:
import torch.nn as nn
import torch
from src.ModelTrainTestManager import ModelTrainTestManager, optimizer_setup
from src.DataManager import DataManager
from src.Models.ResNet import ResNet

Parameters for the data :

In [11]:
data_csv = "data/common_train.csv"
dataFolderPath = "data/common_cropped_train_imgs"

Parameters for the training :

In [12]:
batch_size = 3
learning_rate = 0.01
optimizer_factory = optimizer_setup(torch.optim.Adam, lr=learning_rate)

test_percentage = 0.2
val_percentage = 0.2

exp_name = "HappyWhale"

Parameters for the model :

In [13]:
num_classes=20
in_channels=3
depth=3
option="small"
size=256/2

model = ResNet(num_classes, in_channels, depth, option, size, use_arcface=False)

In [16]:
data_manager = DataManager(data_csv, dataFolderPath, batch_size,test_percentage, val_percentage, verbose=True)

model_trainer = ModelTrainTestManager(model=model,
                                        data_manager = data_manager,
                                        loss_fn=nn.CrossEntropyLoss(),
                                        optimizer_factory=optimizer_factory,
                                        exp_name = exp_name ,
                                        learning_rate=learning_rate,
                                        use_cuda=True)

                image  individual_id
0  000a8f2d5c316a.jpg             12
1  001001f099519f.jpg              2
2  00144776eb476d.jpg             12
3  0024057bbc89a4.jpg              1
4  0028f6fa123686.jpg              9
Dataset size : 1000
Size of validation set : 160
Size of test set : 200
Size of train set : 640


TypeError: __init__() got an unexpected keyword argument 'img_size'

In [15]:
epoch=10
model_trainer.train(epoch)

  0%|          | 0/214 [00:00<?, ?it/s]

Epoch: 1 of 10


  0%|          | 0/214 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x65536 and 262144x20)

In [ ]:
model_trainer.evaluate_on_test_set()
model_trainer.plot_metrics()

Accuracy on the test set: 0.050 %


TypeError: plot_metrics() missing 1 required positional argument: 'path'